# Stellar Blends Classification

### In this notebook we run the un-normalized and normalized datasets through the MuyGPyS classifier (a python classifying function that uses the MuyGPS  Gaussian process hyperparameter estimation method), and compare the resulting accuracies.

**Note:** Must have run `data_normalization.ipynb` to continue.

In [ ]:
from MuyGPyS.optimize.loss import mse_fn, lool_fn, pseudo_huber_fn, looph_fn

In [1]:
# from MuyGPyS import config
# config.update("muygpys_jax_enabled", False)

import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from MuyGPyS.examples.classify import do_classify
from MuyGPyS.gp.deformation import F2, Isotropy
from MuyGPyS.gp.hyperparameter import Parameter, Parameter as ScalarParam
from MuyGPyS.gp.kernels import RBF, Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.loss import LossFn, cross_entropy_fn



/home/eleh/miniconda3/envs/muygps/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


### Read in all flattened data (normalized and un-normalized):

In [2]:
from glob import glob

# read normalized data csv file names from the data directory
norm_data_names = glob('../data/data-norm/max-only/*.csv')
# get rid of "../data/data-norm/"
norm_data_names = [name.split('/')[-1] for name in norm_data_names]
# norm_data_names[:10]

In [3]:
# sort the names by their numbers
norm_data_names.sort(key=lambda x: x.split('_')[1])
norm_data_names

['nthroot_0.03448.csv',
 'nthroot_0.06897.csv',
 'nthroot_0.1034.csv',
 'nthroot_0.1379.csv',
 'nthroot_0.1724.csv',
 'nthroot_0.2069.csv',
 'nthroot_0.2414.csv',
 'nthroot_0.2759.csv',
 'nthroot_0.3103.csv',
 'nthroot_0.3448.csv',
 'nthroot_0.3793.csv',
 'nthroot_0.4138.csv',
 'nthroot_0.4483.csv',
 'nthroot_0.4828.csv',
 'nthroot_0.5172.csv',
 'nthroot_0.5517.csv',
 'nthroot_0.5862.csv',
 'nthroot_0.6207.csv',
 'nthroot_0.6552.csv',
 'nthroot_0.6897.csv',
 'nthroot_0.7241.csv',
 'nthroot_0.7586.csv',
 'nthroot_0.7931.csv',
 'nthroot_0.8276.csv',
 'nthroot_0.8621.csv',
 'nthroot_0.8966.csv',
 'nthroot_0.931.csv',
 'nthroot_0.9655.csv',
 'nthroot_1.0.csv',
 'norm_1.csv',
 'norm_11.csv',
 'norm_2.csv',
 'norm_21.csv',
 'norm_3.csv',
 'norm_31.csv',
 'norm_4.csv',
 'norm_41.csv',
 'norm_5.csv',
 'norm_51.csv',
 'raw_image_data.csv',
 'nthroot_log.csv',
 'nthroot_mm0.03448.csv',
 'nthroot_mm0.06897.csv',
 'nthroot_mm0.1034.csv',
 'nthroot_mm0.1379.csv',
 'nthroot_mm0.1724.csv',
 'nthroot_

### Define a function that generates "one-hot" values.

This essentially just takes our truth labels of 0 and 1, and does the following conversions for use in the classifier:
- 0 to [1., -1.]
- 1 to [-1., 1.]

In [4]:
def generate_onehot_value(values):
    onehot = []
    for val in values:
        if val == 0:
            onehot.append([1., -1.])
        elif val == 1:
            onehot.append([-1., 1.])
    return onehot

### Run the classifier on each dataset

For each dataset (un-normalized and normalized) in `data_files`, this for loop does the following:
- Separate labels from data
- Split up data between training and testing
    - `test_size` is the fraction of the data you want to use for testing, where 0.5 means half of the data is used for testing and half for training.
    - `random_state` makes each dataset get trained and tested on the same number of stars and galaxies.
- Gets the one-hot values for the testing and training labels
- Gets `train` and `test` into the proper format for the classifier, a dictionary with the keys: 
    - 'input': 
    - 'output':
    - 'lookup':
- Does the classification (`do_classify`)
- Computes the accuracy of the classifier for the given dataset, by compairing predicted labels to truth labels.

In [5]:
nn_kwargs_exact = {"nn_method": "exact", "algorithm": "ball_tree"}

nn_kwargs_hnsw = {"nn_method": "hnsw"}

k_kwargs_rbf ={
            "kernel": RBF(
                 deformation=Isotropy(
                     metric=F2,
                 length_scale=Parameter(1.0, (1e-2, 1e2)),
                 ),
            ),
            "noise": HomoscedasticNoise(1e-5),
            }
k_kwargs_mattern= { "kernel": Matern(
             smoothness=ScalarParam(0.5),
             deformation=Isotropy(
                 metric=F2,
                 length_scale=Parameter(1.0, (1e-2, 1e2)),
             ),
         ),
         "noise": HomoscedasticNoise(1e-5),
         }

In [6]:
norm_name = []
my_accuracy = []
for path in tqdm(norm_data_names):
    path1 = '../data/data-norm/max-only/' + path
    data = pd.read_csv(path1,na_values='-')
    data.fillna(0,inplace=True)
    data_label = ''.join(path.split('.')[:2])
    truth_labels = data.iloc[:, 0].values
    image_data = data.iloc[:, 1:].values

    X_train, X_test, y_train, y_test = train_test_split(image_data, truth_labels, test_size=0.2, random_state=42)

    print("=============== ", data_label, " ===============")
    print('Training data:', len(y_train[y_train==0]), 'single stars and', len(y_train[y_train==1]), 'blended stars')
    print('Testing data:', len(y_test[y_test==0]), 'single stars and', len(y_test[y_test==1]), 'blended stars')

    onehot_train, onehot_test = generate_onehot_value(y_train), generate_onehot_value(y_test)

    train = {'input': X_train, 'output': onehot_train, 'lookup': y_train}
    test = {'input': X_test, 'output': onehot_test, 'lookup': y_test}

    print("Running Classifier on", data_label)
    #Switch verbose to True for more output


    muygps, nbrs_lookup, surrogate_predictions = do_classify(
                                test_features=np.array(test['input']), 
                                train_features=np.array(train['input']), 
                                train_labels=np.array(train['output']), 
                                nn_count=30,
                                batch_count=200,
                                loss_fn=cross_entropy_fn,
                                opt_fn=Bayes_optimize,
                                k_kwargs=k_kwargs_mattern,
                                nn_kwargs=nn_kwargs_hnsw,
                                verbose=False)
    predicted_labels = np.argmax(surrogate_predictions, axis=1)
    accur = np.around((np.sum(predicted_labels == np.argmax(test["output"], axis=1))/len(predicted_labels))*100, 3)
    norm_name.append(data_label.split('_')[-1])
    my_accuracy.append(accur)
    print("Total accuracy for", data_label, ":", accur, '%')

===============  nthroot_003448  ===============
Training data: 12073 single stars and 9728 blended stars
Testing data: 3036 single stars and 2415 blended stars
Running Classifier on nthroot_003448
Total accuracy for nthroot_003448 : 77.16 %
===============  nthroot_006897  ===============
Training data: 12073 single stars and 9728 blended stars
Testing data: 3036 single stars and 2415 blended stars
Running Classifier on nthroot_006897
Total accuracy for nthroot_006897 : 78.243 %
===============  nthroot_01034  ===============
Training data: 12073 single stars and 9728 blended stars
Testing data: 3036 single stars and 2415 blended stars
Running Classifier on nthroot_01034
Total accuracy for nthroot_01034 : 77.032 %
===============  nthroot_01379  ===============
Training data: 12073 single stars and 9728 blended stars
Testing data: 3036 single stars and 2415 blended stars
Running Classifier on nthroot_01379
Total accuracy for nthroot_01379 : 77.637 %
===============  nthroot_01724  ===

In [7]:
min(my_accuracy), max(my_accuracy)

(69.584, 82.187)

In [16]:
accura = pd.DataFrame({'norm_name': norm_name, 'accuracy': my_accuracy})
accura.to_csv('../data/muygps-max-only-accuracy.csv', index=False)

accura = pd.read_csv('../data/muygps-max-only-accuracy.csv')   
accura.sort_values(by=['accuracy'], inplace=True)
accura.T

,35,38,34,36,70,17,25,19,28,16,...,32,37,50,59,48,57,55,49,43,41
norm_name,4csv,51csv,31csv,41csv,scalercsv,06207,08966,06897,10,05862,...,21csv,5csv,mm03448,mm06552,mm02759,mm05862,mm05172,mm03103,mm01034,mm003448
accuracy,69.584,70.024,70.666,70.684,73.766,74.096,74.408,74.445,74.629,74.665,...,79.967,80.022,80.04,80.059,81.38,81.636,81.655,81.728,81.912,82.187


In [18]:
accura.nlargest(15, 'accuracy')

,norm_name,accuracy
41,mm003448,82.187
43,mm01034,81.912
49,mm03103,81.728
55,mm05172,81.655
57,mm05862,81.636
48,mm02759,81.380
59,mm06552,80.059
50,mm03448,80.040
37,5csv,80.022
32,21csv,79.967


<u>***Note:*** Each time you run the classifier will result in different accuracies.</u>

### As you can see, all 5 normalization techniques do much better than the un-normalized data, with some performing better than others.

### Things you can try, to see how they affect the classifier accuracy:
- Play around with different values of `test_size`. What does testing on more or less data do?
- Play around with different parameters that are passed to `do_classify`. Start with `nn_count` and `embed_dim`(For what those arguments are, and a full list of all of the arguments you can pass to do_classify, look at the function `do_classify` in `/MuyGPyS/examples/classify.py`).
- Try generating more cutouts using `generating_ZTF_cutouts_from_ra_dec.ipynb`. How does having more testing and training data affects the classifier?
- Play around with the parameters used to make the cutouts. What happens if you remove blend cuts? Can the classifier classify blends? What is you increase the seeing limit? Can the classifier classify images with bad atmoshperic quality?

<hr style="border:2px solid gray"> </hr>

## <u>**Optional Step:**</u>
### Running each dataset through the classifier multiple times, testing and training on varying amounts of data, different random states, and plotting the accuracy outcomes

- Each time you run the following steps, you change:
    - `test_size`: This is used in `train_test_split`, and changes the size of the testing and training datasets, which effects the accuracy of the classifier.
    - `random_state`: This is used in `train_test_split`, and changes the ratio of how many stars-to-galaxies get tested on.
- You can set how many times to run the classifier with varying test sizes and random states by setting `num_runs`, and you can manually change the test_size values by editing `test_size_values`.

In [8]:
test_size_values = [.2, .25, .33, .4, .5, .75]
num_runs = 3

In [9]:
# def run_classifier(image_data, truth_labels, test_size, state):
#     X_train, X_test, y_train, y_test = train_test_split(image_data, truth_labels, test_size=test_size, random_state=state)
#     onehot_train, onehot_test = generate_onehot_value(y_train), generate_onehot_value(y_test)
#     train = {'input': X_train, 'output': onehot_train, 'lookup': y_train}
#     test = {'input': X_test, 'output': onehot_test, 'lookup': y_test}
#     #Switch verbose to True for more output
#     muygps, nbrs_lookup, surrogate_predictions= do_classify(
#                         test_features=np.array(test['input']),
#                         train_features=np.array(train['input']), 
#                         train_labels=np.array(train['output']), 
#                         nn_count=20,
#                         batch_count=200,
#                         loss_fn=cross_entropy_fn,
#                         opt_fn=Bayes_optimize,
#                         k_kwargs=k_kwargs_mattern,
#                         nn_kwargs=nn_kwargs_hnsw, 
#                         verbose=False) 
#     predicted_labels = np.argmax(surrogate_predictions, axis=1)
#     accuracy = (np.sum(predicted_labels == np.argmax(test["output"], axis=1))/len(predicted_labels))*100
#     return accuracy

In [10]:
# accuracies = pd.DataFrame({'test_size': test_size_values})

# # Setting progress bar for each time the classifier will be run during this step
# pbar = tqdm(total=len(norm_data_names)*num_runs*len(test_size_values), desc='Running classifier', leave=True)

# for path in norm_data_names:
#     path1 = '../data/data-norm/max-only/' + path
#     data = pd.read_csv(path1,na_values='-')
#     data.fillna(0,inplace=True)
#     data_label = ''.join(path.split('.')[:2])
#     truth_labels = data.iloc[:, 0].values
#     image_data = data.iloc[:, 1:].values
#     all_acc_dataset = []
#     for test_size in test_size_values:
#         acc = []
#         idx = 1
#         while idx <= num_runs:
#             accuracy = run_classifier(image_data, truth_labels, test_size, state=random.randint(0, 10000))
#             acc.append(accuracy)
#             pbar.update(1)
#             idx += 1
#         avg_acc = np.average(acc)
#         all_acc_dataset.append(avg_acc)
#     temp_df = pd.DataFrame({str(data_label): all_acc_dataset})
#     accuracies = pd.concat([accuracies, temp_df], axis=1)
# display(accuracies)

In [11]:
# plt.figure(figsize=(12,4))

# for path in norm_data_names:
#     path1 = '../data/data-norm/max-only/' + path
#     data = pd.read_csv(path1,na_values='-')
#     data.fillna(0,inplace=True)
#     data_label = ''.join(path.split('.')[:2])
#     # data_label = 'Normalized {} {}'.format(*path.split('_')[:2])
#     plt.plot(accuracies['test_size'].values, accuracies[data_label].values, label=data_label)

# plt.title("MuyGPs Stellar Blending 2-class")    
# plt.legend(fontsize=10)   
# plt.tick_params(labelsize=10)
# plt.xlabel("Test size (as a ratio to full data size)", fontsize=10)
# plt.ylabel("Accuracy [%]", fontsize=10)
# plt.savefig("muygps_.png")
# plt.show()

In [12]:
# accuracies.to_csv("max-only-accuracy.csv")